In [4]:
import requests
from requests import Response
aqi_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'

res: Response = requests.get(aqi_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

#自訂的欄位驗証
from pydantic import BaseModel, Field, field_validator

class Site(BaseModel):
    site_name: str = Field(alias='sitename')
    county: str
    aqi: int
    status: str
    pm25: float = Field(alias='pm2.5')
    so2: float
    co: float

    @field_validator('pm25', 'so2', 'co', mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        if value == '':
            return '0.0'
        else:
            return value

class AQI(BaseModel):
    records: list[Site]

data: AQI = AQI.model_validate_json(res.text)

下載成功


In [5]:
all_sites:list[dict] = data.model_dump()['records']
all_sites #python的基本資料結構

[{'site_name': '屏東(枋山)',
  'county': '屏東縣',
  'aqi': 33,
  'status': '良好',
  'pm25': 5.0,
  'so2': 0.2,
  'co': 0.13},
 {'site_name': '大甲（日南國小）',
  'county': '臺中市',
  'aqi': 22,
  'status': '良好',
  'pm25': 7.0,
  'so2': 0.1,
  'co': 0.21},
 {'site_name': '新北(樹林)',
  'county': '新北市',
  'aqi': 41,
  'status': '良好',
  'pm25': 16.0,
  'so2': 0.4,
  'co': 0.35},
 {'site_name': '屏東（琉球）',
  'county': '屏東縣',
  'aqi': 25,
  'status': '良好',
  'pm25': 8.0,
  'so2': 0.0,
  'co': 0.04},
 {'site_name': '臺南（麻豆）',
  'county': '臺南市',
  'aqi': 24,
  'status': '良好',
  'pm25': 9.0,
  'so2': 0.6,
  'co': 0.18},
 {'site_name': '高雄（湖內）',
  'county': '高雄市',
  'aqi': 21,
  'status': '良好',
  'pm25': 5.0,
  'so2': 1.5,
  'co': 0.15},
 {'site_name': '彰化（員林）',
  'county': '彰化縣',
  'aqi': 36,
  'status': '良好',
  'pm25': 17.0,
  'so2': 1.0,
  'co': 0.13},
 {'site_name': '大城',
  'county': '彰化縣',
  'aqi': 29,
  'status': '良好',
  'pm25': 7.0,
  'so2': 0.6,
  'co': 0.04},
 {'site_name': '富貴角',
  'county': '新北市',
  'aqi'

In [6]:
import pandas as pd
df1 = pd.DataFrame(all_sites)
df1

,site_name,county,aqi,status,pm25,so2,co
0,屏東(枋山),屏東縣,33,良好,5.0,0.2,0.13
1,大甲（日南國小）,臺中市,22,良好,7.0,0.1,0.21
2,新北(樹林),新北市,41,良好,16.0,0.4,0.35
3,屏東（琉球）,屏東縣,25,良好,8.0,0.0,0.04
4,臺南（麻豆）,臺南市,24,良好,9.0,0.6,0.18
...,...,...,...,...,...,...,...
995,頭份,苗栗縣,26,良好,9.0,1.2,0.13
996,新竹,新竹市,29,良好,7.0,0.5,0.18
997,竹東,新竹縣,44,良好,9.0,0.4,0.16
998,湖口,新竹縣,48,良好,20.0,0.2,0.19


In [12]:
df1[df1['aqi']<=16]

,site_name,county,aqi,status,pm25,so2,co
875,恆春,屏東縣,16,良好,5.0,0.8,0.07
